# DATA PREPROCESSING - 
This notebook cleans the data present in the csv files and handles missing data, if any.

### CLEANING
- Stripping all unnecessary characters. Eg: 'price' column in "books.csv" contains values with "-", "USD" and "*" that needs to be removed for analysis
- Correcting datatypes of all columns and making them consistent
- converting dates into datetime objects for pandas operations
- Removing all leading and trailing whitespaces, and multiple whitespaces in between words in all the columns.

### Data Imputation
- Removing rows with missing values would not be ideal - we will loose important data
- Names, street addresses, book names, authors and dates cannot be imputed
- City and region names can be imputed since we have been hired by a local library. However, since all libraries are unique, the zipcodes can have a unique default code

In [89]:
# Package imports

import numpy as np
import pandas as pd
import re
from ast import literal_eval
from collections import Counter

In [64]:
# Import and read all files using Pandas
books_df = pd.read_csv("data/books.csv")

In [65]:
books_df[:5]

,id,title,authors,publisher,publishedDate,categories,price,pages
0,hVFwAAAAQBAJ,Ogilvy on Advertising,['David Ogilvy'],Vintage,2013-09-11,['Social Science'],72.99,320
1,bRY9AAAAYAAJ,Foreign Publications for Advertising American ...,['United States. Bureau of Foreign and Domesti...,NaN,1913,['Advertising'],469.99,654
2,ZapAAAAAIAAJ,Advertising and the Public Interest,"['John A. Howard', 'James Hulbert']",NaN,1973,['Advertising'],372.0,784
3,A-HthMfF5moC,Profitable Advertising,NaN,NaN,1894,['Advertising'],240.99USD,559
4,4Z9JAAAAMAAJ,Report of the Federal Trade Commission on Dist...,['United States. Federal Trade Commission'],NaN,1944,['Government publications'],539.0,757


### Data Cleaning

Closely looking at the data in the dataframes, some column's data needs cleaning. For example, the 'price' column in 'Books_df' contains unnecessary characters like '-', 'USD' and '*' mentioned before and after the actual numbers. Hence, those needs to be stripped down to just have numerical values for analysis.

In [66]:
def clean_data(data, only_keep="nums"):
    if only_keep == "nums": # Keep only digits and dots (dots is for price only)
        data = str(data)
        cleaned_price = ''.join([char for char in data if char.isdigit() or char == '.'])
    if cleaned_price == "":
        return np.nan
    return float(cleaned_price)

In [67]:
books_df['price'] = books_df['price'].apply(clean_data)
books_df['pages'] = books_df['pages'].apply(clean_data)

In [68]:
books_df['price'].describe()

count    238.000000
mean     302.476513
std      153.690459
min        5.990000
25%      191.122500
50%      303.995000
75%      396.240000
max      721.000000
Name: price, dtype: float64

In [69]:
books_df['pages'].describe()

count     240.000000
mean      587.762500
std       164.169719
min       124.000000
25%       478.750000
50%       582.500000
75%       704.000000
max      1154.000000
Name: pages, dtype: float64

In [70]:
def get_dtypes(df, column):
    data_types = set()
    for index, row in df.iterrows():
        types = type(row[column])
        data_types.add(types)
        if isinstance(row[column], float):
            print(row[column])
    
    print(data_types)

In [71]:
books_df["publishedDate"] = pd.to_datetime(books_df["publishedDate"], format="mixed")

In [91]:
categories = books_df['categories'].to_list()

all_categories = []

for cat in categories:
    if not pd.isnull(cat):
        cat = literal_eval(cat)
        # print(type(cat))
        for item in cat:
            # print(type(item))
            all_categories.append(item)
        # break

all_categories = dict(Counter(all_categories))
all_categories = dict(sorted(all_categories.items(), key=lambda item: item[1], reverse=True)[:5])
all_categories

{'Business & Economics': 26,
 'Advertising': 16,
 'Medicine': 15,
 'Science': 9,
 'Technology & Engineering': 7}

In [92]:
books_df.to_csv("data/cleaned_books.csv")

## CHECKOUTS DF

In [218]:
checkouts_df = pd.read_csv("data/checkouts.csv")

In [219]:
checkouts_df[:10]

,id,patron_id,library_id,date_checkout,date_returned
0,-xFj0vTLbRIC,b071c9c68228a2b1d00e6f53677e16da,225-222@5xc-jtz-hkf,2019-01-28,2018-11-13
1,HUX-y4oXl04C,8d3f63e1deed89d7ba1bf6a4eb101373,223-222@5xc-jxr-tgk,2018-05-29,2018-06-12
2,TQpFnkku2poC,4ae202f8de762591734705e0079d76df,228-222@5xc-jtz-hwk,2018-11-23,2019-01-24
3,OQ6sDwAAQBAJ,f9372de3c8ea501601aa3fb59ec0f524,23v-222@5xc-jv7-v4v,2018-01-15,2018-04-25
4,7T9-BAAAQBAJ,2cf3cc3b9e9f6c608767da8d350f77c9,225-222@5xc-jtz-hkf,2018-12-31,1804-01-23
5,iGoXAQAAMAAJ,80f93362e97d9f6108ea781e57739214,22c-222@5xc-jwj-pvz,2018-04-07,1815-08-11
6,CW-7tHAaVR0C,dd9f34e9d65126a2b02003d8ac60aaa4,22c-222@5xc-jwj-pvz,2018-01-10,2018-02-04
7,Cr74DwAAQBAJ,69a2fbbf7aaad8ac0729e55d85858b0e,23v-222@5xc-jv7-v4v,2106-02-26,2018-12-10
8,t1e3BWziAc8C,3b85b2c7b424618f533329018e9a11d5,222-222@5xc-jv5-nt9,2018-06-23,2018-07-14
9,2mtCAAAAYAAJ,365ab35282c5d61b319ff9cc3628e624,228-222@5xc-jtz-hwk,2018-08-07,2018-08-25


In [220]:
checkouts_df.count()

id               2000
patron_id        2000
library_id       2000
date_checkout    1935
date_returned    1942
dtype: int64

In [221]:
def clean_dates(date):
    # Remove leading/trailing whitespace and special characters
    if pd.isna(date):
        # print("-----isna----", date)
        return np.nan
    
    else:
        date = str(date)
        print("before----", date)
        cleaned_date = re.sub(r"[^\d]", '', date) # regex only keeps the digits and removes all other characters
        # # Add dashes in YYYYMMDD format
        # if len(cleaned_date) == 8 and cleaned_date.isdigit():
        #     cleaned_date = f"{cleaned_date[:4]}-{cleaned_date[4:6]}-{cleaned_date[6:]}"
        pd_dt_date = pd.to_datetime(cleaned_date, errors='coerce')
        # print("after----", pd_dt_date)
        return pd_dt_date.date()

In [ ]:
checkouts_df['date_checkout'] = checkouts_df['date_checkout'].apply(clean_dates)
checkouts_df['date_returned'] = checkouts_df['date_returned'].apply(clean_dates)

In [224]:
checkouts_df.to_csv("data/cleaned_checkouts.csv")

## CUSTOMERS DF

In [36]:
customers_df = pd.read_csv("data/customers.csv")

In [37]:
# customers_df["name"]
get_dtypes(customers_df, "zipcode")

nan
{<class 'str'>, <class 'float'>}


In [38]:
def clean_zipcodes(code):
    # Remove leading/trailing whitespace and special characters
    if pd.isna(code):
        # print("-----isna----", date)
        return np.nan
    
    else:
        code = str(code)
        code = code.replace(".0", '')
        # print("before----", code)
        cleaned_code = re.sub(r"[^\d]", '', code) # regex only keeps the digits and removes all other characters
        # print("after----", pd_dt_date)
        return cleaned_code

In [39]:
customers_df['zipcode'] = customers_df['zipcode'].apply(clean_zipcodes)

In [331]:
# for i, row in customers_df.iterrows():
#     print(row["zipcode"])

In [40]:
customers_df["birth_date"] = pd.to_datetime(customers_df["birth_date"], format="mixed")

In [41]:
def clean_all(level):    
    if pd.isna(level):
        return np.nan
    
    level = level.strip().lower()
    clean_level = re.sub(r'\s+', ' ', level).title() # Replace multiple spaces with a single space and convert to title case
    return clean_level
    

In [42]:
column_names = ['name', 'street_address', 'city', 'state', 'gender', 'education', 'occupation']
for column in column_names:
    customers_df[column] = customers_df[column].apply(clean_all)


In [43]:
customers_df[:10]

,id,name,street_address,city,state,zipcode,birth_date,gender,education,occupation
0,df83ec2d0d409395c0d8c2690cfa8b67,Cynthia Barnfield,44 Ne Meikle Pl,Portland,Oregon,97213,2009-09-10,Female,High School,NaN
1,6aec7ab2ea0d67161dac39e5dcabd857,Elizabeth Smith,7511 Se Harrison St,Portland,Oregon,97215,1956-12-15,Female,College,Blue Collar
2,0c54340672f510fdb9d2f30595c1ab53,Richard Pabla,1404 Se Pine St,Portland,Oregon,97214,1960-12-18,Male,College,Education & Health
3,f0d9ce833ddc1f73c1e0b55bdebf012e,Charles Baker,12271 N Westshore Dr,Portland,Oregon,97217,2105-07-19,Male,Graduate Degree,Sales
4,3720379163f6b46944db6c98c0485bfd,Ronald Lydon,5321 Ne Skyport Way,NaN,Oregon,97218,1961-03-14,Male,Graduate Degree,Blue Collar
5,8380f7f05a183e9b17f133d5e8a78707,Katherine Bax,4106 Ne 69Th Ave,Portland,Oregon,97218,1990-10-06,Female,Graduate Degree,Tech
6,5b54076ff51088b8ce4f2c3692b79d66,Christine Hermreck,1856 Ne 84Th Ave,Portland,Oregon,97220,2009-05-27,Female,College,Education & Health
7,0b585fb06695db3347b6e340f7f897bf,Jeffrey Snyder,11440 Se Aquila St,Happy Valley,Oregon,97086,1956-10-17,Male,High School,Sales
8,bfc7e857c666777a3ee53a9ed576ccdb,Marco Royals,1731 Nw Everett St,Portland,Oregon,97209,1980-09-13,Male,Graduate Degree,Education & Health
9,7fd53f66a6d4cd12875487af4008bb08,Travis Hardy,636 Se 154Th Ave,Portland,Oregon,97233,1803-04-20,Male,College,Sales


In [44]:
customers_df.to_csv("data/cleaned_customers.csv")

## LIBRARIES DF

In [337]:
libraries_df = pd.read_csv("data/libraries.csv")

In [338]:
column_names = ["name","street_address","city","region"]
for column in column_names:
    libraries_df[column] = libraries_df[column].apply(clean_all)

In [341]:
libraries_df['region'] = libraries_df['region'].apply(lambda x: np.nan if pd.isna(x) else x.upper())

In [343]:
libraries_df['postal_code'] = libraries_df['postal_code'].apply(clean_zipcodes)

In [344]:
libraries_df

,id,name,street_address,city,region,postal_code
0,226-222@5xc-kc4-fpv,Multnomah County Library Capitol Hill,10723 Sw Capitol Hwy,Portland,OR,97219
1,23v-222@5xc-jv7-v4v,Multnomah County Library Northwest,2300 Nw Thurman St,NaN,OR,NaN
2,222-222@5xc-jvf-skf,Multnomah County Library St Johns,7510 N Charleston Ave,Portland,OR,97203
3,227-222@5xc-jww-btv,Multnomah County Library Hillsdale,1525 Sw Sunset Blvd,Portland,OR,97239
4,22d-222@5xc-kcy-8sq,Multnomah County Library Sellwood Moreland,7860 Se 13Th Ave,Portland,OR,97202
5,223-222@5xc-jxr-tgk,Multnomah County Library Woodstock,6008 Se 49Th Ave,Portland,OR,97206
6,zzw-224@5xc-jwv-2rk,Multnomah County Library Central,801 Sw 10Th Ave,Portland,NaN,97205
7,zzw-223@5xc-jv7-ct9,Friends Of The Multnomah County Library,522 Sw 5Th Ave,NaN,OR,97204
8,226-222@5xc-jxj-7yv,Multnomah County Library Belmont,1038 Se Cesar E Chavez Blvd,Portland,OR,97214
9,zzw-222@5xc-knn-c5z,Multnomah County Library Holgate,7905 Se Holgate Blvd,Portland,OR,NaN


In [345]:
libraries_df.to_csv("data/cleaned_libraries.csv")

## Handling missing values

In [45]:
customers_df = pd.read_csv("data/cleaned_customers.csv")
customers_df

,Unnamed: 0,id,name,street_address,city,state,zipcode,birth_date,gender,education,occupation
0,0,df83ec2d0d409395c0d8c2690cfa8b67,Cynthia Barnfield,44 Ne Meikle Pl,Portland,Oregon,97213.0,2009-09-10,Female,High School,NaN
1,1,6aec7ab2ea0d67161dac39e5dcabd857,Elizabeth Smith,7511 Se Harrison St,Portland,Oregon,97215.0,1956-12-15,Female,College,Blue Collar
2,2,0c54340672f510fdb9d2f30595c1ab53,Richard Pabla,1404 Se Pine St,Portland,Oregon,97214.0,1960-12-18,Male,College,Education & Health
3,3,f0d9ce833ddc1f73c1e0b55bdebf012e,Charles Baker,12271 N Westshore Dr,Portland,Oregon,97217.0,2105-07-19,Male,Graduate Degree,Sales
4,4,3720379163f6b46944db6c98c0485bfd,Ronald Lydon,5321 Ne Skyport Way,NaN,Oregon,97218.0,1961-03-14,Male,Graduate Degree,Blue Collar
...,...,...,...,...,...,...,...,...,...,...,...
1995,1995,ae55f0b71b8b8e91945cd9a91b6e45ee,Joe Roberts,7331 Ne Killingsworth St,Portland,NaN,97218.0,1955-05-23,Male,Others,Business & Finance
1996,1996,07fe407cc889ea21a8bdc04c305960b1,Matthew Coniglio,1908 Nw Harborside Dr,Vancouver,Washington,98660.0,1975-11-10,Male,Others,Business & Finance
1997,1997,9a2194fcd4f0f326f0ca334450e16a93,Earl Grier,22 Ne Graham St,Portland,Oregon,97212.0,2007-10-02,Male,Others,Education & Health
1998,1998,01a598a05c48fdd18461d6411f51a109,Rogelio Richmann,7000 Ne Airport Way,Portland,Oregon,97218.0,2001-02-19,Male,College,Business & Finance


In [46]:
code_dict = {}

for idx, row in customers_df.iterrows():
    code_dict[row["zipcode"]] = code_dict.get(row["zipcode"], 0) + 1
    
print(code_dict)

# Could have used Counter instead
# df.value_counts() - Jupyter truncating the final output

{97213.0: 71, 97215.0: 31, 97214.0: 82, 97217.0: 186, 97218.0: 76, 97220.0: 93, 97086.0: 29, 97209.0: 42, 97233.0: 20, 97034.0: 13, 98664.0: 5, 97007.0: 3, 97206.0: 82, 97229.0: 13, 97224.0: 11, 97230.0: 46, 97232.0: 59, 97210.0: 67, 97227.0: 43, 97211.0: 100, 97219.0: 94, 97035.0: 20, 97203.0: 85, 97216.0: 30, 97045.0: 8, 97202.0: 100, 97267.0: 14, 97201.0: 47, 97212.0: 82, 97266.0: 57, 97080.0: 2, 97225.0: 31, 97008.0: 9, 97239.0: 77, 97221.0: 36, 97005.0: 7, 97236.0: 21, 97222.0: 63, 97068.0: 9, 98685.0: 2, 97205.0: 17, 97223.0: 26, 97062.0: 9, 98661.0: 14, 97204.0: 8, 97231.0: 17, 98682.0: 2, 98660.0: 9, 97089.0: 2, 98663.0: 7, 97015.0: 3, 97024.0: 1, 98662.0: 2, 98665.0: 3, 97124.0: 1, 97030.0: 2, 97060.0: 1, 98683.0: 2, 97027.0: 1, 97006.0: 3, 97078.0: 1, 98684.0: 2, nan: 1}


In [47]:
customers_df["state"].value_counts()

state
Oregon        1851
Washington      47
Name: count, dtype: int64

In [48]:
customers_df["city"].isna().value_counts()

city
False    1906
True       94
Name: count, dtype: int64

No missing vlaues in the Customers Dataframe. Hence no imputation is required.

In [57]:
library_df = pd.read_csv("data/cleaned_libraries.csv")
library_df

,Unnamed: 0.1,Unnamed: 0,id,name,street_address,city,region,postal_code
0,0,0,226-222@5xc-kc4-fpv,Multnomah County Library Capitol Hill,10723 Sw Capitol Hwy,Portland,OR,97219.0
1,1,1,23v-222@5xc-jv7-v4v,Multnomah County Library Northwest,2300 Nw Thurman St,Portland,OR,NaN
2,2,2,222-222@5xc-jvf-skf,Multnomah County Library St Johns,7510 N Charleston Ave,Portland,OR,97203.0
3,3,3,227-222@5xc-jww-btv,Multnomah County Library Hillsdale,1525 Sw Sunset Blvd,Portland,OR,97239.0
4,4,4,22d-222@5xc-kcy-8sq,Multnomah County Library Sellwood Moreland,7860 Se 13Th Ave,Portland,OR,97202.0
5,5,5,223-222@5xc-jxr-tgk,Multnomah County Library Woodstock,6008 Se 49Th Ave,Portland,OR,97206.0
6,6,6,zzw-224@5xc-jwv-2rk,Multnomah County Library Central,801 Sw 10Th Ave,Portland,OR,97205.0
7,7,7,zzw-223@5xc-jv7-ct9,Friends Of The Multnomah County Library,522 Sw 5Th Ave,Portland,OR,97204.0
8,8,8,226-222@5xc-jxj-7yv,Multnomah County Library Belmont,1038 Se Cesar E Chavez Blvd,Portland,OR,97214.0
9,9,9,zzw-222@5xc-knn-c5z,Multnomah County Library Holgate,7905 Se Holgate Blvd,Portland,OR,NaN


Since, we're talking about a local library, the city and region can be the same. Moreover, the addresses of these libraries are unique and hence Null values can be defaulted to a random code. 

In [58]:
default_code = 10000 # 5 digit placeholder code - will increment this and assign to each library

code_series = library_df["postal_code"]
updated_codes = []
for code in code_series:
    if pd.isnull(code):
            updated_codes.append(default_code)
            default_code += 1
    else:
        updated_codes.append(code)
            

library_df["postal_code"] = pd.Series(updated_codes)

In [59]:
library_df

,Unnamed: 0.1,Unnamed: 0,id,name,street_address,city,region,postal_code
0,0,0,226-222@5xc-kc4-fpv,Multnomah County Library Capitol Hill,10723 Sw Capitol Hwy,Portland,OR,97219.0
1,1,1,23v-222@5xc-jv7-v4v,Multnomah County Library Northwest,2300 Nw Thurman St,Portland,OR,10000.0
2,2,2,222-222@5xc-jvf-skf,Multnomah County Library St Johns,7510 N Charleston Ave,Portland,OR,97203.0
3,3,3,227-222@5xc-jww-btv,Multnomah County Library Hillsdale,1525 Sw Sunset Blvd,Portland,OR,97239.0
4,4,4,22d-222@5xc-kcy-8sq,Multnomah County Library Sellwood Moreland,7860 Se 13Th Ave,Portland,OR,97202.0
5,5,5,223-222@5xc-jxr-tgk,Multnomah County Library Woodstock,6008 Se 49Th Ave,Portland,OR,97206.0
6,6,6,zzw-224@5xc-jwv-2rk,Multnomah County Library Central,801 Sw 10Th Ave,Portland,OR,97205.0
7,7,7,zzw-223@5xc-jv7-ct9,Friends Of The Multnomah County Library,522 Sw 5Th Ave,Portland,OR,97204.0
8,8,8,226-222@5xc-jxj-7yv,Multnomah County Library Belmont,1038 Se Cesar E Chavez Blvd,Portland,OR,97214.0
9,9,9,zzw-222@5xc-knn-c5z,Multnomah County Library Holgate,7905 Se Holgate Blvd,Portland,OR,10001.0


In [60]:
library_df = library_df.fillna({"region":"OR", "city":"Portland"})


In [61]:
library_df

,Unnamed: 0.1,Unnamed: 0,id,name,street_address,city,region,postal_code
0,0,0,226-222@5xc-kc4-fpv,Multnomah County Library Capitol Hill,10723 Sw Capitol Hwy,Portland,OR,97219.0
1,1,1,23v-222@5xc-jv7-v4v,Multnomah County Library Northwest,2300 Nw Thurman St,Portland,OR,10000.0
2,2,2,222-222@5xc-jvf-skf,Multnomah County Library St Johns,7510 N Charleston Ave,Portland,OR,97203.0
3,3,3,227-222@5xc-jww-btv,Multnomah County Library Hillsdale,1525 Sw Sunset Blvd,Portland,OR,97239.0
4,4,4,22d-222@5xc-kcy-8sq,Multnomah County Library Sellwood Moreland,7860 Se 13Th Ave,Portland,OR,97202.0
5,5,5,223-222@5xc-jxr-tgk,Multnomah County Library Woodstock,6008 Se 49Th Ave,Portland,OR,97206.0
6,6,6,zzw-224@5xc-jwv-2rk,Multnomah County Library Central,801 Sw 10Th Ave,Portland,OR,97205.0
7,7,7,zzw-223@5xc-jv7-ct9,Friends Of The Multnomah County Library,522 Sw 5Th Ave,Portland,OR,97204.0
8,8,8,226-222@5xc-jxj-7yv,Multnomah County Library Belmont,1038 Se Cesar E Chavez Blvd,Portland,OR,97214.0
9,9,9,zzw-222@5xc-knn-c5z,Multnomah County Library Holgate,7905 Se Holgate Blvd,Portland,OR,10001.0


In [62]:
library_df.to_csv("data/cleaned_libraries.csv")